In [ ]:
import gym_electric_motor as gem
import gym
import tensorflow as tf
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from gym.wrappers import FlattenObservation
#from tf_agents.environments.wrappers import FlattenObservationsWrapper
from tf_agents.agents.dqn import dqn_agent
from tf_agents.utils import common

from gym_electric_motor.visualization import MotorDashboard, ConsolePrinter
from gym_electric_motor.physical_systems import ConstantSpeedLoad

import numpy as np

/home/praneeth/anaconda3/envs/tfagents/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/home/praneeth/anaconda3/envs/tfagents/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/home/praneeth/anaconda3/envs/tfagents/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
# visualization=MotorDashboard(plots = ['i_sq', 'i_sd', 'reward']),
    
#                # parameterize the PMSM
#                motor_parameter=motor_parameter,
#                #converter = converter,
    
#                # update the limitations of the state space
#                limit_values=limit_values,
#                nominal_values=nominal_values,
               
#                # define the DC link voltage
#                u_sup=u_sup, 
               
#                # define the speed at which the motor is operated
#                load='ConstSpeedLoad',
#                load_initializer = {'states': {'omega': 1000 * np.pi / 30,},
#                                    'interval': [[-4000*2*np.pi/60, 4000*2*np.pi/60]],
#                                    'random_init': 'uniform',
#                    },
#                #gem.physical_systems.ConstantSpeedLoad(omega_fixed=1000 * np.pi / 30), 
               
#                # define the duration of one sampling step
#                tau=1e-5,
               
#                # turn off terminations via limit violation and parameterize the reward function
#                reward_function=gem.reward_functions.WeightedSumOfErrors(observed_states=['i_sq', 'i_sd'], 
#                                                                         reward_weights={'i_sq': 1, 'i_sd': 1},
#                                                                         #constraint_monitor = SqdCurrentMonitor(),
#                                                                         gamma = gamma,
#                                                                         reward_power=1
#                                                                        ),
#                # define the reference generator
#                reference_generator=rg,
    
#                # define a numerical solver of adequate accuracy
#                ode_solver='euler'

In [ ]:
q_generator = gem.reference_generators.WienerProcessReferenceGenerator(reference_state='i_sq')
d_generator = gem.reference_generators.WienerProcessReferenceGenerator(reference_state='i_sd')
rg = gem.reference_generators.MultipleReferenceGenerator([q_generator, d_generator])

# Change of the default motor parameters.
motor_parameter = dict(
    r_s=15e-3, l_d=0.37e-3, l_q=1.2e-3, psi_p=65.6e-3, p=3, j_rotor=0.06
)
limit_values = dict(
    i=160*1.41,
    omega=12000 * np.pi / 30,
    u=450
)
nominal_values = {key: 0.7 * limit for key, limit in limit_values.items()}
u_sup = 400
gamma = 0.99

class SqdCurrentMonitor:
    """
    monitor for squared currents:
    
    i_sd**2 + i_sq**2 < 1.5 * nominal_limit 
    """
    def __init__(self, ):
        self.I_SD_IDX = 5
        self.I_SQ_IDX = 6
    
    def __call__(self, state, observed_states, k, physical_system):
        self.I_SD_IDX = physical_system.state_names.index('i_sd')
        self.I_SQ_IDX = physical_system.state_names.index('i_sq')
        # normalize to limit_values, since state is normalized
        nominal_values = physical_system.nominal_state / abs(physical_system.limits)
        limits = 1.5 * nominal_values
        # calculating squared currents as observed measure 
        sqd_currents = state[self.I_SD_IDX]**2 + state[self.I_SQ_IDX]**2
        
        return (sqd_currents > limits[self.I_SD_IDX] or sqd_currents > limits[self.I_SQ_IDX])

gym_env_kwargs = {'visualization': MotorDashboard(plots = ['i_sq', 'i_sd', 'reward']),
            # parameterize the PMSM
               'motor_parameter' : motor_parameter,
               'limit_values' : limit_values,
               'nominal_values' : nominal_values,
               'u_sup' : u_sup,
               'load' : ConstantSpeedLoad(omega_fixed=1000 * np.pi / 30),
               'state_filter' :['i_sq', 'i_sd', 'epsilon'],
               'reward_weights' : {'i_sq': 1000, 'i_sd': 1000},
               'reward_power' : 0.5,
               'observed_states' : ['i_sq', 'i_sd'],
           
            #'tau' : 1e-4 ,
            # turn off terminations via limit violation and parameterize the reward function
#             'reward_function' : gem.reward_functions.WeightedSumOfErrors(observed_states=['i_sq', 'i_sd'], 
#                                                                         reward_weights={'i_sq': 1, 'i_sd': 1},
#                                                                         #constraint_monitor = SqdCurrentMonitor(),
#                                                                         gamma = gamma,
#                                                                         reward_power=1
#                                                                        ),
            
            'reference_generator' : rg,
            # define a numerical solver of adequate accuracy
            'ode_solver' : 'euler' #'scipy.solve_ivp'
    
}


In [ ]:
# Alternate way to create a tf compatible gym env

t_env = gym.make(# define a PMSM with continuous action space
               "emotor-pmsm-disc-v1", **gym_env_kwargs)
t_env_f =FlattenObservation(t_env) 

t_py_env = suite_gym.wrap_env(t_env_f, max_episode_steps=10000)
t_train_env = tf_py_environment.TFPyEnvironment(t_py_env)

In [ ]:
env_name = "emotor-pmsm-disc-v1"
env = suite_gym.load(env_name, max_episode_steps=10000, gym_env_wrappers=[FlattenObservation],
                      gym_kwargs = gym_env_kwargs )  #
# #env = FlattenObservation(env)
train_env = tf_py_environment.TFPyEnvironment(env)
print(env.gym)


In [ ]:
train_env.observation_spec()

In [ ]:
# create a neural network:

fc_layer_params = (100,)

q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)


In [ ]:
# Hyper-parameters

num_iterations = 2000 

initial_collect_steps = 1000  
collect_steps_per_iteration = 1 
replay_buffer_max_length = 100000 

batch_size = 64  
learning_rate = 1e-4  
log_interval = 200 

num_eval_episodes = 10 
eval_interval = 1000

In [ ]:
# instantiate DQN agent

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()


In [ ]:
class ShowProgress:
    def __init__(self, total):
        self.counter = 0
        self.total = total
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % 100 == 0:
            print("\r{}/{}".format(self.counter, self.total), end="")

In [ ]:
# create a replay buffer

from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

replay_buffer_observer = replay_buffer.add_batch

In [ ]:
from tf_agents.metrics import tf_metrics

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric(),
]

In [ ]:
# Driver to take action steps in the environment
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver

collect_driver = DynamicStepDriver(
    train_env,
    agent.collect_policy,
    observers=[replay_buffer_observer] + train_metrics) 

In [ ]:
# fill the replay buffer initially with trajectories from a random policy

from tf_agents.policies.random_tf_policy import RandomTFPolicy

initial_collect_policy = RandomTFPolicy(train_env.time_step_spec(),
                                        train_env.action_spec())
init_driver = DynamicStepDriver(
    train_env,
    initial_collect_policy,
    observers=[replay_buffer.add_batch, ShowProgress(2000)],
    num_steps=2000) 
final_time_step, final_policy_state = init_driver.run()

In [ ]:
from tf_agents.trajectories import trajectory

def collect_step(environment, policy, buffer):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
  for _ in range(steps):
    collect_step(env, policy, buffer)

random_policy = RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())
collect_data(train_env, random_policy, replay_buffer, steps=100)

In [ ]:
# dataset is sampled from the replay buffer

dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)


dataset
iterator = iter(dataset)

print(iterator)

In [ ]:
#optional
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

eval_py_env = suite_gym.load(env_name, gym_env_wrappers=[FlattenObservation])
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [ ]:
from tf_agents.utils.common import function

agent.train = common.function(agent.train)
collect_driver.run = function(collect_driver.run)

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

# for _ in range(num_iterations):

#   # Collect a few steps using collect_policy and save to the replay buffer.
#   for _ in range(collect_steps_per_iteration):
#     collect_step(train_env, agent.collect_policy, replay_buffer)

#   # Sample a batch of data from the buffer and update the agent's network.
#   experience, unused_info = next(iterator)
#   train_loss = agent.train(experience).loss

#   step = agent.train_step_counter.numpy()

#   if step % log_interval == 0:
#     print('step = {0}: loss = {1}'.format(step, train_loss))

#   if step % eval_interval == 0:
#     avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
#     print('step = {0}: Average Return = {1}'.format(step, avg_return))
#     returns.append(avg_return)

time_step = None
policy_state = agent.collect_policy.get_initial_state(train_env.batch_size)
#iterator = iter(dataset)
for iteration in range(num_iterations):
    time_step, policy_state = collect_driver.run(time_step, policy_state)
    trajectories, buffer_info = next(iterator)
    train_loss = agent.train(trajectories)
    train_env.render()
    print("\r{} loss:{:.5f}".format(
        iteration, train_loss.loss.numpy()), end="")
    if iteration % 1000 == 0:
        log_metrics(train_metrics)